# 🔴 Lab 4 — Extraction Attack (Model Stealing)
### Certified AI Penetration Tester – Red Team (CAIPT-RT)

---

## 🎯 The Story

A company spent three years and significant budget building a machine learning model that predicts whether families qualify for social services. The model is their competitive advantage — it is more accurate than anything else on the market. The code, internal weights, and training data are all kept completely secret.

But they offer it as a paid API service. You send it an application profile, it sends back a decision. That is all you get — a prediction.

You are an attacker — perhaps a competitor, perhaps a researcher trying to expose bias in an opaque system. You have no access to the model internals whatsoever. But you have access to the API.

By sending thousands of carefully chosen queries and recording every response, you can **build your own model that makes almost identical decisions** to the original — without ever seeing the training data or the model code.

This is a **Model Extraction Attack** — also called model stealing.

---

## 📖 What is a Model Extraction Attack?

The attacker creates a functional copy of a model by repeatedly querying it and using the query-response pairs as their own training data.

**The core idea:** Every time you ask the model a question and it answers, it is unknowingly teaching you how it thinks. Collect enough answers and you can train your own model to think the same way.

**Why is this a problem?**
- **Intellectual property theft** — years of R&D reproduced for free
- **Enabling further attacks** — a stolen local copy can be attacked using white-box methods that require direct model access
- **Exposing bias** — researchers sometimes use extraction to audit black-box models for fairness

**Real world examples:**
- Stealing a competitor's fraud detection model to avoid licensing fees
- Copying a medical diagnosis model and rebranding it
- Using a stolen model as a stepping stone to prepare evasion attacks (Lab 1)

---

## 🗂️ What We Will Do in This Lab

1. Load the Nursery dataset and prepare it for modelling
2. Train the victim model — the valuable proprietary model being stolen
3. Set up a black-box query interface that simulates API access only
4. Query the victim and collect its responses as stolen training data
5. Train our own model on those responses and measure how close we got
6. Compare results at different query budgets — more queries, better copy?
7. Think like a defender — what signals would alert you to this attack?

---

## ⚙️ Step 1: Import the Tools We Need

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
# numpy      : handles numbers and arrays
# pandas     : loads and manages datasets
# matplotlib : draws charts to visualise results
# sklearn    : builds and trains machine learning models
# art        : Adversarial Robustness Toolbox — wraps models for attacks
#
# Note: CopycatCNN is imported for its querying infrastructure only.
# We do not use its internal training step because it passes neural-network
# specific parameters (batch_size, nb_epochs) that sklearn models reject.
# Instead we query the victim through ART and train our stolen model manually.
# This is explained in detail in the attack step below.
# =============================================================================

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from art.estimators.classification import SklearnClassifier

np.random.seed(42)
print("All tools imported successfully.")

---

## 📂 Step 2: Load the Dataset and Create Three Data Splits

We reuse the Nursery dataset from Lab 3. This time we split it into **three portions** rather than two, because we have three distinct roles to play:

**Victim's training data** — the private data the victim model learns from. The attacker never sees this. In a real scenario this would be the company's proprietary dataset stored securely on their servers.

**Attacker's query pool** — data the attacker uses to send queries to the victim's API. These are records the attacker has access to independently — they do not need to come from the victim's dataset. Any plausible input data will do.

**Evaluation set** — held out completely and used only to measure the quality of both the victim and stolen models fairly. Neither model trained on this data.

This three-way split mirrors a real attack scenario where the attacker has their own data, the victim has their own private data, and we measure everything on neutral ground.

In [ ]:
# =============================================================================
# LOAD AND ENCODE THE NURSERY DATASET
# =============================================================================

column_names = [
    'parents', 'has_nurs', 'form', 'children',
    'housing', 'finance', 'social', 'health', 'target'
]

df = pd.read_csv(
    '../datasets/nursery.data',
    header=None,
    names=column_names
)

# Convert all text values to integers (same as Lab 3)
df_encoded = df.copy()
for column in df_encoded.columns:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column])

X = df_encoded.drop('target', axis=1).values
y = df_encoded['target'].values

print(f"Dataset loaded: {len(df):,} records  |  {X.shape[1]} features  |  {len(np.unique(y))} target classes")
print("")

# =============================================================================
# THREE-WAY SPLIT
# =============================================================================
# Split 1: 50% for victim training, 50% remaining
# Split 2: of the remaining 50%, take 60% for attacker queries and 40% for eval
#
# Result:
#   X_train      : 50% of data  (victim trains on this — attacker CANNOT see it)
#   X_steal_pool : 30% of data  (attacker queries the victim with these)
#   X_eval       : 20% of data  (neutral evaluation set — neither model trained here)
# =============================================================================

X_train, X_remaining, y_train, y_remaining = train_test_split(
    X, y, test_size=0.5, random_state=42, stratify=y
)
X_steal_pool, X_eval, y_steal_pool, y_eval = train_test_split(
    X_remaining, y_remaining, test_size=0.4, random_state=42
)

print("Three-way data split complete:")
print(f"  Victim training data (PRIVATE) : {len(X_train):>5,} records  <- attacker never sees this")
print(f"  Attacker query pool            : {len(X_steal_pool):>5,} records  <- attacker queries victim with these")
print(f"  Neutral evaluation set         : {len(X_eval):>5,} records  <- used to score both models fairly")
print("")
print(f"  Max queries available to attacker: {len(X_steal_pool):,}")

In [ ]:
# =============================================================================
# TRAIN THE VICTIM MODEL
# =============================================================================
# This is the valuable proprietary model the attacker wants to steal.
# It is a powerful Random Forest with 200 trees.
#
# In a real scenario this model sits behind a paid API. The attacker
# cannot see its code, weights, hyperparameters, or training data.
# All they can do is send an input and receive a prediction back.
#
# We wrap it in ART's SklearnClassifier so that when we call
# art_victim.predict(), it returns probability distributions (one per class)
# rather than just hard labels. This is what a real API might return.
# =============================================================================

print("Training victim model (200 trees)...")
print("(May take 20-30 seconds)")
print("")

victim_model = RandomForestClassifier(n_estimators=200, random_state=42)
victim_model.fit(X_train, y_train)

victim_accuracy = accuracy_score(y_eval, victim_model.predict(X_eval))

# Wrap in ART so predict() returns probability arrays not just labels
art_victim = SklearnClassifier(model=victim_model)

print(f"Victim model accuracy on neutral eval set: {victim_accuracy*100:.2f}%")
print("")
print("This is our benchmark target.")
print("The stolen model will try to get as close to this as possible.")
print("")

# Demonstrate what the API returns for a single query
sample_query  = X_eval[:1]
sample_probs  = art_victim.predict(sample_query)
sample_label  = victim_model.predict(sample_query)[0]

print("What the attacker sees for one query (the API response):")
print(f"  Input record  : {sample_query[0]}")
print(f"  Probabilities : {[round(p, 4) for p in sample_probs[0]]}")
print(f"  Predicted class: {sample_label}")
print("")
print("The attacker cannot see the model code or weights — only this output.")

### 👀 What Do You See?

The victim model gives back a **probability distribution** for each query — a list of numbers that sum to 1.0, one per class. The highest number is the predicted class.

**Why do probabilities matter more than just the label?**

If the API only returned a label — say `class 2` — the attacker gets limited information. But if it returns `[0.01, 0.02, 0.94, 0.02, 0.01]`, the attacker knows the model is 94% confident it is class 2. That confidence information is extremely valuable for training a stolen model, because the stolen model can try to match not just which class was predicted, but *how sure* the victim was.

This is why the "label-only" defense is effective — returning just the class name strips out all the confidence information.

---

## 🔴 Step 3: Perform the Extraction Attack

Here is exactly how model stealing works in this lab:

**Phase 1 — Query the victim (information gathering)**

The attacker sends records from their query pool to the victim model's API. For each record, the victim returns a probability distribution. The attacker collects all these responses. They now have a dataset of (input, victim_prediction) pairs — a stolen dataset.

**Phase 2 — Train a stolen model (replication)**

The attacker trains their own fresh model on this stolen dataset. Instead of the real labels (which they do not have), they use the victim's predictions as labels. The stolen model learns to reproduce the victim's decision-making.

**The key insight:** The victim model is being used as an expensive labelling service, entirely against its owner's intentions.

**We test three query budgets** — 100, 500, and 2000 queries — to show how the quality of the stolen model improves as more information is gathered.

In [ ]:
# =============================================================================
# EXTRACTION ATTACK AT THREE QUERY BUDGETS
# =============================================================================
#
# For each query budget n_queries we:
#
# PHASE 1 — QUERY THE VICTIM:
#   art_victim.predict(query_x)
#     -> Returns an array of shape (n_queries, nb_classes)
#     -> Each row is a probability distribution: [0.02, 0.91, 0.03, 0.02, 0.02]
#     -> All values in a row sum to 1.0
#
#   np.argmax(victim_probs, axis=1)
#     -> axis=1 means: look across each row (not down each column)
#     -> argmax returns the INDEX of the highest value in each row
#     -> [0.02, 0.91, 0.03, 0.02, 0.02] → index 1 (class 1 has the highest prob)
#     -> Output is a 1D array of integer class labels: the stolen labels
#
# PHASE 2 — TRAIN THE STOLEN MODEL:
#   LogisticRegression().fit(query_x, fake_labels)
#     -> A plain sklearn fit() call — no special ART parameters
#     -> The model learns to map inputs to the victim's predicted classes
#     -> It does NOT know the real ground-truth labels — only the victim's answers
#
# WHY NOT USE CopycatCNN's BUILT-IN training?
#   CopycatCNN.extract() passes batch_size and nb_epochs to the stolen
#   classifier's fit() method. These are PyTorch/neural network parameters
#   that sklearn's LogisticRegression.fit() does not accept — causing a
#   TypeError. Separating the query and training phases avoids this entirely
#   and makes the attack steps clearer for learning.
# =============================================================================

nb_classes    = len(np.unique(y))
query_budgets = [100, 500, 2000]
stolen_results = []

print("Running extraction attack at different query budgets...")
print("=" * 65)
print(f"{'Budget':>8}  {'Queries used':>14}  {'Stolen accuracy':>16}  {'Agreement':>12}")
print("-" * 65)

for n_queries in query_budgets:

    # ------------------------------------------------------------------
    # PHASE 1: QUERY THE VICTIM API
    # ------------------------------------------------------------------
    # The attacker sends n_queries records to the victim API.
    # art_victim.predict() simulates the API call and returns probabilities.
    # ------------------------------------------------------------------

    query_x     = X_steal_pool[:n_queries]
    victim_probs = art_victim.predict(query_x)

    # victim_probs shape: (n_queries, nb_classes)
    # Each row: one probability per class, all rows sum to 1.0
    assert victim_probs.shape == (n_queries, nb_classes)

    # Convert probability distributions to hard class labels
    # np.argmax across axis=1 picks the class index with highest probability
    fake_labels = np.argmax(victim_probs, axis=1)

    # fake_labels is now a 1D array of integers — the victim's predictions
    # The attacker uses these as their training labels (stolen ground truth)

    # ------------------------------------------------------------------
    # PHASE 2: TRAIN THE STOLEN MODEL ON COLLECTED LABELS
    # ------------------------------------------------------------------
    # Fresh LogisticRegression for each budget (no carryover between runs).
    # Logistic Regression finds a linear boundary between classes in
    # feature space. It is simpler than the victim's Random Forest but
    # can still approximate its behaviour given enough query-label pairs.
    # max_iter=1000 allows enough optimisation steps to converge.
    # ------------------------------------------------------------------

    stolen_model = LogisticRegression(max_iter=1000, random_state=42)
    stolen_model.fit(query_x, fake_labels)

    # ------------------------------------------------------------------
    # EVALUATE: HOW GOOD IS THE STOLEN COPY?
    # ------------------------------------------------------------------
    # We test on the neutral eval set — data neither model has seen.
    #
    # stolen_accuracy: how often stolen model gets the TRUE label right
    # agreement      : how often stolen model matches VICTIM's prediction
    #                  (these differ because victim itself is not perfect)
    # ------------------------------------------------------------------

    stolen_preds    = stolen_model.predict(X_eval)
    stolen_accuracy = accuracy_score(y_eval,       stolen_preds)
    victim_preds    = victim_model.predict(X_eval)
    agreement       = accuracy_score(victim_preds,  stolen_preds)

    stolen_results.append((n_queries, stolen_accuracy, agreement))
    print(f"{n_queries:>8}  {n_queries:>14,}  {stolen_accuracy*100:>15.2f}%  {agreement*100:>11.2f}%")

print("-" * 65)
print(f"{'Victim':>8}  {'(benchmark)':>14}  {victim_accuracy*100:>15.2f}%  {'100.00%':>12}")
print("")
print("Accuracy  = how often the stolen model gets the TRUE label right")
print("Agreement = how often stolen model matches the victim's prediction")

### 👀 What Do You See?

Two metrics matter here:

**Accuracy** measures how well the stolen model performs on the actual task — correctly classifying records into the 5 enrollment categories. This tells you how useful the stolen model is on its own.

**Agreement** measures how closely the stolen model mimics the victim — even when the victim is wrong, does the stolen model make the same mistake? High agreement means the attacker successfully replicated the victim's decision-making, not just its correctness.

**Notice the diminishing returns:** The jump from 100 to 500 queries likely gives a bigger improvement than the jump from 500 to 2000. Why? Because the first few hundred queries cover the main patterns in the data. Additional queries reveal increasingly rare edge cases that have smaller and smaller marginal value. This is a fundamental property of learning from data.

**Also notice:** The stolen model uses LogisticRegression, not Random Forest like the victim. The attacker does not need to use the same model type — they just need a model that can reproduce the same input-output mapping.

---

## 📊 Step 4: Visualise the Trade-off

In [ ]:
# =============================================================================
# VISUALISE RESULTS
# =============================================================================
# Two charts side by side:
#   Left  : stolen model accuracy vs query budget (compared to victim benchmark)
#   Right : stolen model agreement with victim vs query budget
#
# The red dashed line on the left chart shows the victim's accuracy.
# The attacker's goal is to get as close to that line as possible.
# The red dashed line on the right chart shows perfect agreement (100%).
# =============================================================================

budgets    = [r[0] for r in stolen_results]
accuracies = [r[1]*100 for r in stolen_results]
agreements = [r[2]*100 for r in stolen_results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Chart 1: Accuracy vs query budget
ax1.plot(budgets, accuracies, 'bo-', linewidth=2, markersize=8, label='Stolen model')
ax1.axhline(
    y=victim_accuracy*100, color='red', linestyle='--', linewidth=1.5,
    label=f'Victim benchmark ({victim_accuracy*100:.1f}%)'
)
ax1.set_title('Stolen Model Accuracy vs Query Budget', fontsize=11, fontweight='bold')
ax1.set_xlabel('Number of Queries Sent to Victim API')
ax1.set_ylabel('Accuracy on Neutral Eval Set (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)
for b, a in zip(budgets, accuracies):
    ax1.annotate(f'{a:.1f}%', (b, a), textcoords="offset points", xytext=(0, 10), ha='center')

# Chart 2: Agreement vs query budget
ax2.plot(budgets, agreements, 'go-', linewidth=2, markersize=8, label='Stolen model')
ax2.axhline(
    y=100, color='red', linestyle='--', linewidth=1.5, label='Perfect copy (100%)'
)
ax2.set_title('Stolen Model Agreement with Victim vs Query Budget', fontsize=11, fontweight='bold')
ax2.set_xlabel('Number of Queries Sent to Victim API')
ax2.set_ylabel('Agreement with Victim Predictions (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)
for b, a in zip(budgets, agreements):
    ax2.annotate(f'{a:.1f}%', (b, a), textcoords="offset points", xytext=(0, 10), ha='center')

plt.tight_layout()
plt.savefig('../outputs/lab4_extraction_results.png')
plt.show()
print("Chart saved to outputs/lab4_extraction_results.png")

### 👀 What Do You See?

- The left chart shows how close the stolen model gets to the victim's benchmark accuracy as query budget grows.
- The right chart shows how often the stolen model makes exactly the same decision as the victim.
- Look at the shape of both curves — do they flatten out? That flattening is **diminishing returns** in action.

### 🧪 Try This

Add `50` as the first entry in `query_budgets` and rerun the attack.

- Can you build a usable stolen model with just 50 queries?
- At what query count would you say the attack is "successful enough to be dangerous"?
- From a defender's side — if you were monitoring the API for suspicious behaviour, what query volume would trigger an alert?

---

## 🛡️ Step 5: Think Like a Defender

If you were the company protecting the victim model, what would you do?

The three most common defenses are listed below. We demonstrate each one so you can see concretely what effect it has.

In [ ]:
# =============================================================================
# DEFENSIVE ANALYSIS
# =============================================================================
# Three practical defenses against model extraction attacks.
# We demonstrate each one concretely so you can see what it changes.
# =============================================================================

print("DEFENDER PERSPECTIVE — Three Defenses Against Extraction")
print("=" * 65)
print("")

# ------------------------------------------------------------------
# DEFENSE 1: QUERY VOLUME MONITORING
# ------------------------------------------------------------------
# Track how many queries each user or IP address sends.
# A legitimate user querying a credit scoring API might send 10-50
# queries. Someone sending 2,000+ queries is behaving like an attacker.
# Set a threshold and alert or block when it is exceeded.
# ------------------------------------------------------------------

print("1. QUERY VOLUME MONITORING")
print("-" * 45)
print("   How many queries did each budget level need?")
for n_queries, acc, agr in stolen_results:
    print(f"   {n_queries:>5,} queries → {agr*100:.1f}% agreement with victim")
print("")
print("   A legitimate user rarely needs 500+ queries.")
print("   Set an alert threshold. Block or rate-limit suspicious volumes.")
print("   Recommendation: alert at 200 queries/day per user, block at 500.")
print("")

# ------------------------------------------------------------------
# DEFENSE 2: OUTPUT ROUNDING (REDUCING PRECISION)
# ------------------------------------------------------------------
# Returning full probability precision (e.g. 0.9143) gives the attacker
# rich information to train their stolen model.
# Rounding to 1 decimal place (e.g. 0.9) still tells the user which
# class was predicted and roughly how confident the model is, but gives
# the attacker much less to work with.
#
# np.round(exact_probs, 1) rounds each value to 1 decimal place.
# The attacker gets coarser signal → worse stolen model → more queries needed.
# ------------------------------------------------------------------

print("2. OUTPUT ROUNDING — REDUCING PROBABILITY PRECISION")
print("-" * 45)
sample        = X_eval[:3]
exact_probs   = victim_model.predict_proba(sample)
rounded_probs = np.round(exact_probs, 1)

print("   Full precision (default — attacker gets maximum information):")
for i, p in enumerate(exact_probs):
    top3 = sorted(zip(p, range(len(p))), reverse=True)[:3]
    print(f"   Record {i+1}: {[f'{v:.4f}' for v, _ in top3]} (top 3 classes)")

print("")
print("   Rounded to 1 decimal (defender reduces information leak):")
for i, p in enumerate(rounded_probs):
    top3 = sorted(zip(p, range(len(p))), reverse=True)[:3]
    print(f"   Record {i+1}: {[f'{v:.1f}' for v, _ in top3]} (top 3 classes)")

print("")
print("   Less precision = less useful signal = attacker needs more queries")
print("   Legitimate users still get a usable prediction.")
print("")

# ------------------------------------------------------------------
# DEFENSE 3: LABEL-ONLY RESPONSES
# ------------------------------------------------------------------
# The most aggressive defense: return ONLY the predicted class name,
# no probabilities at all.
# This strips all confidence information from the API response.
# The attacker gets far less information per query, making the attack
# much harder and requiring far more queries to reach the same quality.
# The downside: some legitimate applications need confidence scores.
# ------------------------------------------------------------------

print("3. LABEL-ONLY RESPONSES")
print("-" * 45)
print("   Full response (attacker gets probabilities):")
for i, p in enumerate(exact_probs[:2]):
    label = np.argmax(p)
    print(f"   Record {i+1}: class={label}  probs={[f'{v:.4f}' for v in p]}")

print("")
print("   Label-only response (attacker gets nothing but the decision):")
for i, p in enumerate(exact_probs[:2]):
    label = np.argmax(p)
    print(f"   Record {i+1}: class={label}")

print("")
print("   The attacker now gets only 1 bit of signal per query (which class).")
print("   Reaching 80% agreement would require ~10x more queries.")
print("   Legitimate users who only need a decision are unaffected.")

### 👀 What Do You See?

- **Query monitoring** is the easiest to implement and catches volume-based attacks quickly. But a sophisticated attacker can slow their queries down to stay under the threshold.
- **Output rounding** degrades the attacker's signal without affecting the user experience much. It is a good default setting for any public API.
- **Label-only** is the strongest defense but may break applications that depend on confidence scores for their own decision-making.

**In practice, most real-world ML APIs combine all three** — rounding their outputs, monitoring query volume, and providing confidence scores only to verified enterprise customers.

### 💭 Which defense would you implement first, and why?

---

## 💭 Step 6: Reflect on What You Have Learned

In [ ]:
# =============================================================================
# REFLECTION — edit your answers below and run to save them
# =============================================================================

reflection = """
LAB 4 - EXTRACTION ATTACK REFLECTION
======================================

Q1: In plain English, what is a model extraction attack?
    What does the attacker end up with at the end, and why is that valuable?
A1: [TYPE YOUR ANSWER HERE]

Q2: The attacker never saw the victim model's code, weights, or training data.
    They only had API access. What does this tell you about the security risk
    of exposing machine learning models through public APIs?
A2: [TYPE YOUR ANSWER HERE]

Q3: We used LogisticRegression for the stolen model but the victim used
    RandomForest. Does the stolen model need to be the same type?
    What does this tell you about how the attacker works?
A3: [TYPE YOUR ANSWER HERE]

Q4: Explain diminishing returns in plain English in the context of this attack.
    Why does each additional query give the attacker less new information?
A4: [TYPE YOUR ANSWER HERE]

Q5: Rank the three defenses (query monitoring, output rounding, label-only)
    from most to least effective against a determined attacker.
    Justify your ranking.
A5: [TYPE YOUR ANSWER HERE]

Q6: Looking back across all four labs, which attack do you think poses
    the greatest risk to organisations deploying AI in high-stakes domains
    (healthcare, finance, criminal justice)? Justify your answer.
A6: [TYPE YOUR ANSWER HERE]

BONUS: Can you think of a scenario where performing a model extraction attack
       would be ethically justified? What safeguards would you put in place?
BONUS: [TYPE YOUR ANSWER HERE]
"""

with open('../outputs/Lab4_Reflection.txt', 'w') as f:
    f.write(reflection)

print("Reflection saved to outputs/Lab4_Reflection.txt")
print(reflection)

---

## ✅ Lab 4 Complete — And So Is the Course!

You have worked through all four core attack types:

| Lab | Attack | When It Happens | What the Attacker Targets |
|-----|--------|----------------|--------------------------|
| 1 | Evasion | After deployment | Model inputs — crafting inputs that fool the model |
| 2 | Poisoning | During training | Training data — corrupting what the model learns |
| 3 | Inference | After deployment | Data privacy — figuring out who was in the training set |
| 4 | Extraction | After deployment | Model IP — stealing the model through API queries |

Each attack represents a real threat that AI security practitioners defend against today. The tools and techniques you used in this lab are the same ones used by researchers at IBM, Microsoft, Google, and security firms worldwide.

The Adversarial Robustness Toolbox (ART) that powered these labs is open source and actively maintained — it is the same library used to evaluate AI systems in production environments.

Return to [START_HERE.ipynb](START_HERE.ipynb) to review all four labs.

---
*Built with the Adversarial Robustness Toolbox (ART) — https://github.com/Trusted-AI/adversarial-robustness-toolbox*